# Python で気軽に化学・化学工学
# 第 8 章 モデル y = f(x) を構築して、新たなサンプルの y を推定する
## 8.8 ダブルクロスバリデーション (Double Cross-Validation, DCV)
### サポートベクター回帰 (Support Vector Regression, SVR)

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

### 沸点のデータセット (descriptors_8_with_boiling_point.csv)
Hall and Story が収集した[沸点のデータセット](https://pubs.acs.org/doi/abs/10.1021/ci960375x)。294 個の化合物について、沸点 (Boiling Point) が測定されており、8 つの分子記述子 (特徴量) で化学構造が数値化されています。特徴量は、分子量 (MolWt)、水素原子以外の原子で計算された分子量 (HeavyAtomMolWt)、価電子の数 (NumValenceElectrons)、水素原子以外の原子の数 (HeavyAtomCount)、窒素原子と酸素原子の数 (NOCount)、水素原子と炭素原子以外の原子の数 (NumHeteroatoms)、回転可能な結合の数 (NumRotatableBonds)、環の数 (RingCount) です。

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('descriptors_8_with_boiling_point.csv', index_col=0, header=0) # 沸点のデータセットの読み込み

In [ ]:
x = dataset.iloc[:, 1:] # 記述子を 説明変数 x とします

In [ ]:
y = dataset.iloc[:, 0] # 沸点を目的変数 y とします

In [ ]:
estimated_y_in_outer_cv = y.copy() # 外側の CV による y の推定結果を格納する変数

In [ ]:
estimated_y_in_outer_cv # 念のため確認。実際の y と同じものになっています

In [ ]:
outer_fold_number = 10 # 外側の CV における fold 数

In [ ]:
fold_number = 5 # 内側の CV における fold 数

外側の CV の分割

In [ ]:
indexes = [] # fold の番号を格納する変数

In [ ]:
for sample_number in range(x.shape[0]):
    indexes.append(sample_number % outer_fold_number)

In [ ]:
indexes # 念のため確認

参考1 : for 文で list の変数を作成するとき、以下のリスト内包表記を用いることでコードがシンプルになり、また実行時間が短縮されます

In [ ]:
indexes = [sample_number % outer_fold_number for sample_number in range(x.shape[0])]

参考2 : 以下のようにすると for 文を使わずに `indexes` を準備できます

In [ ]:
import numpy as np # NumPy のインポート
from numpy import matlib
min_number = x.shape[0] // outer_fold_number
mod_number = x.shape[0] % outer_fold_number
indexes = np.matlib.repmat(np.arange(outer_fold_number), 1, min_number).ravel()
if mod_number != 0:
    indexes = np.r_[indexes, np.arange(mod_number)]

`indexes` をシャッフル

In [ ]:
import numpy as np # NumPy のインポート

In [ ]:
np.random.seed(99) # 再現性のため乱数の種を固定
fold_index_in_outer_cv = np.random.permutation(indexes) # シャッフル
np.random.seed() # 乱数の種の固定を解除

In [ ]:
fold_index_in_outer_cv # 念のため確認

ガウシアンカーネルを用いた SVR で DCV

In [ ]:
nonlinear_svr_cs = 2 ** np.arange(-5, 11, 1.0) # C の候補

In [ ]:
nonlinear_svr_cs # 念のため確認

In [ ]:
nonlinear_svr_epsilons = 2 ** np.arange(-10, 1, 1.0) # εの候補

In [ ]:
nonlinear_svr_epsilons # 念のため確認

In [ ]:
nonlinear_svr_gammas = 2 ** np.arange(-20, 11, dtype=float) # γの候補

In [ ]:
nonlinear_svr_gammas # 念のため確認

In [ ]:
from sklearn.model_selection import KFold # 内側の CV の分割の設定に使用

In [ ]:
fold = KFold(n_splits=fold_number, shuffle=True, random_state=9) # 内側の CV の分割の設定

ハイパーパラメータの最適化のための準備

In [ ]:
from sklearn.svm import SVR # SVR の実行に使用
from sklearn.model_selection import GridSearchCV # グリッドサーチに使用
from scipy.spatial.distance import cdist # トレーニングデータにおけるユークリッド距離に使用

DCV の実行

In [ ]:
# 外側の CV
for fold_number_in_outer_cv in range(outer_fold_number):
    print(fold_number_in_outer_cv + 1, '/', outer_fold_number)
    # トレーニングデータとテストデータに分割
    x_train = x.iloc[fold_index_in_outer_cv != fold_number_in_outer_cv, :]
    y_train = y.iloc[fold_index_in_outer_cv != fold_number_in_outer_cv]
    x_test = x.iloc[fold_index_in_outer_cv == fold_number_in_outer_cv, :]
    # 特徴量の標準化 (オートスケーリング)
    autoscaled_x_train = (x_train - x_train.mean(axis=0)) / x_train.std(axis=0,ddof=1)
    autoscaled_x_test = (x_test - x_train.mean(axis=0)) / x_train.std(axis=0,ddof=1)
    autoscaled_y_train = (y_train - y_train.mean()) / y_train.std(ddof=1)
    # グラム行列の分散の最大化によるガウシアンカーネルにおける γ の最適化
    square_of_euclidean_distance = cdist(autoscaled_x_train, autoscaled_x_train, metric='sqeuclidean')
    variance_of_gram_matrix = [] # この変数にグラム行列の分散を入れていきます
    for nonlinear_svr_gamma in nonlinear_svr_gammas:
        gram_matrix = np.exp(- nonlinear_svr_gamma * square_of_euclidean_distance)
        variance_of_gram_matrix.append(gram_matrix.var(ddof=1))
    optimal_nonlinear_svr_gamma = nonlinear_svr_gammas[variance_of_gram_matrix.index(max(variance_of_gram_matrix))] # グラム行列の分散が最大となるγ
    # 内側の CV による ε の最適化
    model_for_cross_validation = SVR(kernel='rbf', C=3, gamma=optimal_nonlinear_svr_gamma)
    gs_cv = GridSearchCV(model_for_cross_validation, {'epsilon': nonlinear_svr_epsilons}, cv=fold)
    gs_cv.fit(autoscaled_x_train, autoscaled_y_train)
    optimal_nonlinear_svr_epsilon = gs_cv.best_params_['epsilon']
    # 内側の CV による C の最適化
    model_for_cross_validation = SVR(kernel='rbf', epsilon=optimal_nonlinear_svr_epsilon, gamma=optimal_nonlinear_svr_gamma)
    gs_cv = GridSearchCV(model_for_cross_validation, {'C': nonlinear_svr_cs}, cv=fold)
    gs_cv.fit(autoscaled_x_train, autoscaled_y_train)
    optimal_nonlinear_svr_c = gs_cv.best_params_['C']
    # 内側の CV による γ の最適化
    model_for_cross_validation = SVR(kernel='rbf', epsilon=optimal_nonlinear_svr_epsilon, C=optimal_nonlinear_svr_c)
    gs_cv = GridSearchCV(model_for_cross_validation, {'gamma': nonlinear_svr_gammas}, cv=fold)
    gs_cv.fit(autoscaled_x_train, autoscaled_y_train)
    optimal_nonlinear_svr_gamma = gs_cv.best_params_['gamma']
    # トレーニングデータを用いたモデル構築
    model = SVR(kernel='rbf', C=optimal_nonlinear_svr_c, epsilon=optimal_nonlinear_svr_epsilon, gamma=optimal_nonlinear_svr_gamma) # SVRモデルの宣言
    model.fit(autoscaled_x_train, autoscaled_y_train) # SVRモデル構築
    # テストデータの推定
    estimated_y_test = model.predict(autoscaled_x_test) # テストデータの推定
    estimated_y_test = estimated_y_test * y_train.std(ddof=1) + y_train.mean() #元のスケールに戻す
    estimated_y_in_outer_cv[fold_index_in_outer_cv==fold_number_in_outer_cv] = estimated_y_test # 推定結果を格納

In [ ]:
estimated_y_in_outer_cv # 念のため確認

DCV における y の実測値 vs. 推定値プロット、r<sup>2</sup>, MAE

In [ ]:
estimated_y_in_outer_cv = pd.DataFrame(estimated_y_in_outer_cv) # DataFrame 型に変換

In [ ]:
estimated_y_in_outer_cv.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_in_outer_cv # 念のため確認

In [ ]:
estimated_y_in_outer_cv.to_csv('estimated_y_dcv.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y, estimated_y_in_outer_cv.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y.max(), estimated_y_in_outer_cv.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y.min(), estimated_y_in_outer_cv.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

 r<sup>2</sup>, MAE の計算

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y, estimated_y_in_outer_cv) # r2

In [ ]:
metrics.mean_absolute_error(y, estimated_y_in_outer_cv) # MAE

自分のデータセットをお持ちの方は、そのデータセットでも今回の内容を確認してみましょう。